In [1]:
import os
import requests
from bs4 import BeautifulSoup
import re

In [2]:
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    print(local_filename)
    return local_filename

In [3]:
DAILY_FOLDER = "./pdf_data/"
ES_REPORT_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/situacionActual.htm"
PDF_BASE_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/"

In [4]:
def get_file_names(url):
    try:
        req_page = requests.get(url)
    except Exception as e:
        raise Exception(f"Could not get web page content: {e}")
    
    re_pdf = re.compile(r"(documentos/.+?\.pdf)")

    pdf_paths = list(set(re_pdf.findall(req_page.text)))
    return pdf_paths

In [6]:
pdf_paths = get_file_names(ES_REPORT_URL)

Exception: Could not get web page content: HTTPSConnectionPool(host='www.mscbs.gob.es', port=443): Max retries exceeded with url: /profesionales/saludPublica/ccayes/alertasActual/nCov-China/situacionActual.htm (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'ssl3_read_bytes', 'sslv3 alert handshake failure')])")))

In [ ]:
pdf_paths = [PDF_BASE_URL+i for i in pdf_paths if "Informacion_inicial_alerta" not in i]

In [ ]:
for pdf_path in pdf_paths:
        pdf_name = pdf_path.split("/")[-1]
        pdf_path_get = requests.get(pdf_path)
        with open(
            os.path.join(DAILY_FOLDER, pdf_name),
            'wb'
        ) as f:
            f.write(pdf_path_get.content)

## Last PDFS

In [14]:
for i in range(30,39):
    path = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/documentos/Actualizacion_{}_COVID-19.pdf'.format(i+1)
    download_file(path)

Actualizacion_31_COVID-19.pdf
Actualizacion_32_COVID-19.pdf
Actualizacion_33_COVID-19.pdf
Actualizacion_34_COVID-19.pdf
Actualizacion_35_COVID-19.pdf
Actualizacion_36_COVID-19.pdf
Actualizacion_37_COVID-19.pdf
Actualizacion_38_COVID-19.pdf


In [26]:
lst_files = [file for file in os.listdir() if 'Act' in file]
if os.path.isdir("./pdf_data/"):
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_data/{}".format(f))
else:
    os.mkdir("./pdf_data/")
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_data/{}".format(f))